# Model Building

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from typing import List

Set a predetermined seed so all our results can be replicated

In [4]:
RANDOM_SEED = 1337

# Preprocessing

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
full_data = pd.read_csv('Dataset/clean_data.csv', index_col='Customer_ID')

In [7]:
#Separate target column with rest of the data
churn_col = full_data['churn'].copy()
full_data = full_data.drop('churn',axis=1)

Since we're finished with EDA in the other notebook, we can start splitting the data into training and testing sets. For models that need validation, we will utilize k-fold CV later on. 

In [20]:
#Separate data for training and testing with 80% for training and 20% testing
#Uses our preselected random seed to results are reproducible 
raw_x_train, raw_x_test, y_train, y_test = train_test_split(
    full_data,
    churn_col,
    test_size=0.2,
    random_state=RANDOM_SEED
)

In [9]:
with open('columnDescriptions.json','r') as f:
    col_desc = json.load(f)
    
#Shortened descriptions with elipses for plot titles
#Only retains first 20 characters of description then appends with elipses
short_col_desc = dict(zip(
    col_desc.keys(),
    map(lambda desc: 
        desc if len(desc)<20 else f'{desc[:20]}...', col_desc.values()
    )
))

We will use several different algorithms then compare their performance afterwards to determine which is the best to use. The algorithm we will use are: 
- Logistic Regression
- K Nearest Neighbor Classifier
- Random Forest
- XGBoost
- LightGBM

Since the implementations selected for the above algorithms have differing aptitudes for missing values and normalization, we will need different preprocessing pipelines of the data. For example, XGBoost and LightGBM can handle nan values (XGBoost learns whether to split nan values during training, while LightGBM allocates nan values to reduce loss afterwards) while the sklearn implementations of Logisitic regression, KNN classifier, and random forest cannot. 

In [10]:
from model_utils import PipelineFactory

In [11]:
pf = PipelineFactory(full_data)
impute_normalize_pipe = pf.create_pipe(impute=True,normalize=True)
impute_pipe = pf.create_pipe(impute=True,normalize=False)
ohe_pipe = pf.create_pipe(impute=False,normalize=False)

# Model building

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgbm

## Logistic Regression

In [13]:
x_train = impute_normalize_pipe.fit_transform(raw_x_train)
x_test = impute_normalize_pipe.transform(raw_x_test)

In [18]:
# L2 regularized logistic regression
# 5 default chosen regularization strength
# 5 fold CV (80% training 20% validation)
log_reg = LogisticRegression(
    random_state=RANDOM_SEED,
    class_weight='balanced'
)

In [108]:
from sklearn.decomposition import PCA

In [109]:
pca = PCA()

In [21]:
log_reg.fit(x_train,y_train)

c:\Users\JeffW\Anaconda3\envs\telecomChurn\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced', random_state=1337)